<sub>&copy; 2020 Neuralmagic, Inc., Confidential // [Neural Magic Evaluation License Agreement](https://neuralmagic.com/evaluation-license-agreement/)</sub> 

# PyTorch Transfer Learning with Adam Optimizer

This notebook provides a step-by-step walkthrough for downloading a recalibrated model from the Neural Magic Model Repo and using it for transfer learning. You will:
- Set up the environment
- Select a model*
- Set up the model and dataset
- Perform transfer learning
- Export to [ONNX](https://onnx.ai/)
 
\* Models available in the Neural Magic Model Repo are called out in this notebook. You can familiarize yourself with the models from which you can transfer learn.


Reading through this notebook will be reasonably quick to gain an intuition for what is happening. Rough time estimates for transfer learning are given. Note that training with the PyTorch CPU implementation will be much slower than a GPU:
- 30 minutes on a GPU
- 3 hours on a laptop CPU

## Background
Neural networks can take a long time to train. Model optimization techniques such as [model pruning](https://towardsdatascience.com/pruning-deep-neural-network-56cae1ec5505) may be necessary to achieve both performance and optimizing goals. However, these model optimizations can involve many trials and errors due to a large number of hyperparameters. Fortunately, in the computer vision and natural language space, pruned (sparsified) neural networks [transfer learn well](https://towardsdatascience.com/a-comprehensive-hands-on-guide-to-transfer-learning-with-real-world-applications-in-deep-learning-212bf3b2f27a), allowing end users to get faster time to value with their deep learning deployments without having to start from scratch.

To make it easier to use pruned models, [Neural Magic](https://neuralmagic.com/) is actively:
- Creating pruned versions of popular models and datasets
- Thoroughly testing these models  with the Neural Magic Inference Engine to ensure performance
- Updating the Neural Magic Repo with these models and datasets

## Before you begin…
Be sure to read through the README found in the Neural Magic ML Tooling package.



## Step 1 - Setting Up the Environment

In this step, Neural Magic checks your environment setup to ensure the rest of the notebook will flow smoothly.
Before running, install the neuralmagicML package into the system using the following at the parent of the package directory:

`pip install neuralmagicML-python/ `


In [ ]:
notebook_name = "transfer_learning_adam_pytorch"
print("checking setup for {}...".format(notebook_name))

# filter because of tensorboard future warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

try:
    # make sure neuralmagicML is installed
    import neuralmagicML
except Exception as ex:
    raise Exception(
        "please install neuralmagicML using the setup.py file before continuing"
    )
    
from neuralmagicML.utilsnb import check_pytorch_notebook_setup
check_pytorch_notebook_setup()

## Step 2 - Selecting a Model

Repositories may hold many models, so a simple UI is provided to make this selection process easier. Within the UI, ﬁlters can be applied for models trained in/on speciﬁc domains or datasets. Each network architecture listed will also include options for the dataset it was trained on and the type. The type refers to how the models were trained and/or recalibrated, speciﬁcally:
- base - baseline model, trained generally as in the original paper
- recal - a recalibrated model that is recalibrated to the point of fully recovering the baseline model’s metrics
- recal-perf - a recalibrated model that is recalibrated for performance to the point of recovering 99% of the baseline model’s metrics


In [ ]:
from neuralmagicML.utilsnb import ModelSelectWidgetContainer

print("Creating ui...")
container = ModelSelectWidgetContainer(["pytorch"], ["imagenet"])
display(container.create())

## Model and Dataset Setup

By default, we use the [Imagewoof](https://github.com/fastai/imagenette) dataset to transfer learn to (a dataset consisting of 10 classes of dogs). This dataset is used to show how to transfer learn on a simple dataset quickly. If you would like to try out transfer learning on your own dataset, replace the appropriate lines with your own:
- `num_classes = 10`
- `class_type = "single"`
- `train_dataset = ImagewoofDataset(dataset_root, train=True)`
- `val_dataset = ImagewoofDataset(dataset_root, train=False)`

More information for creating and working with PyTorch datasets can be found [here](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html). Take care to keep the variable names the same, as the rest of the notebook is set up according to those.

In [ ]:
import os
from neuralmagicML.pytorch.datasets import ImagewoofDataset, ImagenetteSize
from neuralmagicML.pytorch.models import ModelRegistry
from neuralmagicML.utils import clean_path

#######################################################
# Define the number of classes to transfer learn to below
#######################################################
num_classes = 10
class_type = "single"  # use single for softmax output, multi for sigmoid output
print(
    "transfer learning to {} classes and class_type {}".format(num_classes, class_type)
)

repo_model = container.selected_model
print("\nloading model {} ...".format(repo_model.root_path))
model = ModelRegistry.create(
    repo_model.registry_key,
    pretrained=repo_model.desc,
    num_classes=num_classes,
    class_type=class_type,
)
model_name = model.__class__.__name__
input_shape = ModelRegistry.input_shape(repo_model.registry_key)
input_size = input_shape[-1]
print(model)

#######################################################
# Define your train and validation datasets below
#######################################################
print("\nloading train dataset...")
train_dataset = ImagewoofDataset(
    train=True, dataset_size=ImagenetteSize.s320, image_size=input_size
)
print(train_dataset)

print("\nloading val dataset...")
val_dataset = ImagewoofDataset(
    train=False, dataset_size=ImagenetteSize.s320, image_size=input_size
)
print(val_dataset)

## Step 4 - Performing Transfer Learning

Now that the model and datasets are chosen and set up, you will begin transfer learning from the given model onto the dataset. The library to enable this is designed to be easily plugged into nearly any training setup for PyTorch. In the cell block below is an example of how an integration looks. The implementation here trains all layers in the selected model. If you do not wish to do that, you can disable specific layers with standard PyTorch code. Note that only four lines are needed to be able to integrate fully.
- Create a `ConstantKSModifier()`. This keeps the sparsity the same for any sparsiﬁed layers.
- Create a `ScheduledModifierManager()`. This is used in combination with the `ConstantKSModifier` and the `ScheduledOptimizer`
- Create a `ScheduledOptimizer()`. This updates the PyTorch objects that modify the training process. It wraps the original optimizer that was used to modify the training process/graph, and should be used in place of that (`optimizer.step()` must be called on ScheduledOptimizer and not the original).
- Call into the `ScheduledOptimizer` for `epoch_start()` and `epoch_end()` while training. These calls mark when an epoch has started and after training when an epoch has ended, respectively.

Once the training objects are created (optimizer, loss function, etc.), a `ConstantKSModifier`, `ScheduledModifierManager`, and `ScheduledOptimizer` are instantiated. Almost all logging and updates are done through TensorBoard for this notebook. The use of TensorBoard is entirely optional. Finally, regular training and testing code for PyTorch is used to go through the process.

Note, for convenience a TensorBoard instance is launched in the cell below pointed at `localhost`. If you are running this notebook on a remote server, then you will need to update TensorBoard accordingly.


In [ ]:
import math
from tqdm import auto
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from neuralmagicML.pytorch.utils import (
    CrossEntropyLossWrapper,
    TopKAccuracy,
    ModuleTrainer,
    ModuleTester,
    TensorBoardLogger,
)
from neuralmagicML.utils import create_unique_dir, clean_path

# setup device, data loaders, loss, optimizer
device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 128
train_data_loader = DataLoader(
    train_dataset, batch_size, shuffle=True, pin_memory=True, num_workers=8
)
val_data_loader = DataLoader(
    val_dataset, batch_size, shuffle=False, pin_memory=True, num_workers=8
)
loss = CrossEntropyLossWrapper(extras={"top1acc": TopKAccuracy(1)})
optim = Adam(model.parameters())
print("device:{} batch_size:{} loss:{}".format(device, batch_size, loss))

tensorboard_model_path = create_unique_dir(
    os.path.join(".", "tensorboard-logs", notebook_name, model_name)
)
loggers = [TensorBoardLogger(tensorboard_model_path)]
print("logging at {}".format(tensorboard_model_path))


#######################################################
# First lines required for transfer learning from a sparse model in PyTorch
#######################################################
from neuralmagicML.pytorch.recal import (
    ScheduledModifierManager,
    ScheduledOptimizer,
    ConstantKSModifier,
)
manager = ScheduledModifierManager([ConstantKSModifier(params="__ALL__")])
optim = ScheduledOptimizer(
    optim,
    model,
    manager,
    steps_per_epoch=math.ceil(len(train_dataset) / batch_size),
    loggers=loggers,
)
print("created modifier, manager, and optimizer")

# we use prewritten trainers and testers to make the code more concise
trainer = ModuleTrainer(model, device, loss, optim, loggers=loggers)
tester = ModuleTester(model, device, loss, loggers=loggers)
model = model.to(device)

# startup tensorboard
%load_ext tensorboard
%tensorboard --logdir ./tensorboard-logs

# run initial validation for comparison
tester.run_epoch(val_data_loader, epoch=-1, show_progress=False)

#######################################################
# Final lines required for recalibrating a model in PyTorch
#######################################################
num_epochs = 20
for epoch in auto.tqdm(range(num_epochs), desc="transfer learning"):
    trainer.run_epoch(train_data_loader, epoch, show_progress=False)
    tester.run_epoch(val_data_loader, epoch, show_progress=False)

# delete so all modifiers are cleaned up before exporting
del optim
print("training completed")

## Step 5 - Exporting to ONNX

Now that the model is fully recalibrated, you need to export it to an ONNX format, which is the format used by the Neural Magic Inference Engine. For PyTorch, exporting to ONNX is natively supported. In the cell block below a convenience class, `ModuleExporter()`, is used to handle exporting.

Once the model is saved as an ONNX ﬁle, it is ready to be used for inference with Neural Magic.


In [ ]:
from neuralmagicML.utils import clean_path
from neuralmagicML.pytorch.utils import ModuleExporter

print("Exporting to onnx...")
export_path = clean_path(os.path.join(".", notebook_name, model_name))
exporter = ModuleExporter(model, export_path)
for batch in val_data_loader:
    sample_input = batch[0]
    break
exporter.export_onnx(sample_input)
print("Exported onnx to {}".format(export_path))

## Next Step

Run your model (ONNX file) through the Neural Magic Inference Engine. The following is an example of code that you can run in your Python console. Be sure to enter your ONNX file path and batch size.

```
from neuralmagic import create_model
model = create_model(onnx_file_path=’some/path/to/model.onnx’, batch_size=1)
inp = [numpy.random.rand(1, 3, 224, 224).astype(numpy.float32)]
out = model.forward(inp)
print(out)
```